In [1]:
from get_embedding import get_embedding
from diffusion import *
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.colors import ListedColormap

import os
import glob
import random

In [3]:
# Todo
# Set Parameters:

n_neighbors=100
step=(60,60)

grid=(30,30)
dt = 0.001
t_total = 10000
n_repeats = 10

In [4]:
detail_result_path = '/Users/pengzhizhang/Documents/test_data/Gastrulation/velocity_result'
output_path = detail_result_path
detail_files = glob.iglob(os.path.join(detail_result_path, 'combined_detail_2000_genes_with_bin_and_smooth.csv'))
lcd = list()
for f in detail_files:
    load_cellDancer_temp = pd.read_csv(f)
    load_cellDancer_temp.rename(columns = {'Unnamed: 0':'cellIndex'}, inplace = True)
    load_cellDancer_temp = load_cellDancer_temp.sort_values(by = ['gene_name', 'cellIndex'], ascending = [True, True])
    lcd.append(load_cellDancer_temp)
load_cellDancer = pd.concat(lcd)

In [5]:
load_cellDancer

,cellIndex,gene_name,s0,u0,s1,u1,alpha,beta,gamma,cost,clusters,cellID,embedding1,embedding2
2465800,0,0610010F05Rik,0.312842,0.113658,0.336885,0.090187,0.062994,0.072669,0.115461,0.051916,Blood progenitors 2,cell_363,3.460521,15.574629
2465801,1,0610010F05Rik,0.246910,0.069799,0.254983,0.056689,0.061463,0.074228,0.113148,0.051916,Blood progenitors 2,cell_382,2.490433,14.971734
2465802,2,0610010F05Rik,0.168916,0.049196,0.176206,0.041304,0.063350,0.074962,0.110840,0.051916,Blood progenitors 2,cell_385,2.351203,15.267069
2465803,3,0610010F05Rik,0.098369,0.023708,0.099891,0.022295,0.064143,0.075953,0.108538,0.051916,Blood progenitors 2,cell_393,5.899098,14.388825
2465804,4,0610010F05Rik,0.116754,0.049414,0.132035,0.041732,0.066686,0.074841,0.109400,0.051916,Blood progenitors 2,cell_398,4.823139,15.374831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931595,12324,Zswim7,0.239490,0.151429,0.261638,0.144773,0.037389,0.029550,0.189151,0.036571,Erythroid3,cell_139318,8.032358,7.603037
4931596,12325,Zswim7,0.320673,0.072177,0.319053,0.072185,0.063542,0.039163,0.179510,0.036571,Erythroid3,cell_139321,10.352904,6.446736
4931597,12326,Zswim7,0.211783,0.068838,0.220460,0.069057,0.064744,0.040553,0.178445,0.036571,Erythroid3,cell_139326,9.464873,7.261099
4931598,12327,Zswim7,0.331291,0.119762,0.338619,0.115295,0.045222,0.032560,0.186174,0.036571,Erythroid3,cell_139327,9.990495,7.243880


In [ ]:
gene_choice=list(set(load_cellDancer.gene_name))
n_neighbors=100
step=(60,60)

embedding, sampling_ixs, velocity_embedding = get_embedding(
    load_raw_data=load_raw_data, 
    load_cellDancer=load_cellDancer, 
    gene_list=gene_choice,
    mode="gene",
    n_neighbors=n_neighbors,
    step=step)

fig, ax = plt.subplots(figsize=(6,6))
plt.quiver(embedding[sampling_ixs, 0],embedding[sampling_ixs, 1],
          velocity_embedding[:,0], velocity_embedding[:,1], color='Blue')
plt.show()

In [ ]:
cell_embedding, normalized_embedding = embedding_normalization(
    embedding[sampling_ixs], embedding, mode='minmax', NORM_ALL_CELLS=True)

velocity = vel_normalization(velocity_embedding, mode='max')

vel_mesh, grid_density, cell_grid_idx, cell_grid_coor, all_grid_idx, all_grid_coor = generate_grid(cell_embedding, normalized_embedding, velocity, steps=(20,20))

# sanity check
plot_mesh_velocity(vel_mesh)


plt.rcParams['figure.figsize']=(6,6)
plt.imshow(grid_density.T, interpolation=None, origin='lower',cmap="Greys")
mpl.rcParams.update(mpl.rcParamsDefault)


plt.show()


In [ ]:
import time

start_time = time.time()
paths = run_diffusion(cell_embedding, vel_mesh, grid_density, 0.005, 10000, 
                      1e-5, False, n_repeats = 20, n_jobs = mp.cpu_count()-1) 
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
newPaths = purge_end_state_stuttering(paths, cell_embedding)
traj_displacement = np.array([compute_trajectory_displacement(ipath) for ipath in newPaths])
# sorted from long to short
order = np.argsort(traj_displacement)[::-1]
sorted_traj = newPaths[order]
traj_displacement=traj_displacement[order]


path_clusters = list()
cell_clusters = list()
path_clusters, cell_clusters = clustering(path_clusters, cell_clusters, 
    sorted_traj, traj_displacement, 0.4, 0, nkeep=-1)

In [ ]:
path = path_clusters[0][0]

from scipy.interpolate import BSpline
from scipy.interpolate import CubicSpline
ptime=np.array(range(len(path)))
cs = CubicSpline(ptime, path)

xs = np.linspace(0, len(path), 1000)

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(cell_embedding[:,0], cell_embedding[:,1], color='grey', alpha=0.3)
ax.scatter(path[:, 0], path[:, 1], s=10, label='data')
ax.scatter(cs(xs)[:, 0], cs(xs)[:, 1], s=10, label='spline')
ax.axes.set_aspect('equal')
ax.legend(loc='upper right')
plt.xlim([0.8, 0.9])
plt.ylim
plt.show()

In [ ]:
import importlib,sys
importlib.reload(sys.modules['diffusion'])
from diffusion import *
all_cell_time = compute_all_cell_time(embedding, cell_embedding, 
                          path_clusters, cell_clusters, 
                          vel_mesh, cell_grid_idx, grid_density, 
                          sampling_ixs, step)
